In [1]:
import pandas as pd

test = pd.read_csv('test.csv')
autoru = pd.read_csv('autoru_cleared.csv')
%matplotlib inline

In [2]:
color_dict = {
    'чёрный': 'black',
    'белый': 'white',
    'серый': 'gray',
    'серебристый': 'silver',
    'синий': 'blue',
    'коричневый': 'gray',
    'красный': 'red',
    'зелёный': 'green',
    'бежевый': 'brown',
    'голубой': 'blue',
    'пурпурный': 'red',
    'золотистый': 'other',
    'фиолетовый': 'violet',
    'жёлтый': 'yellow',
    'оранжевый': 'orange',
    'розовый': 'other',
    'other': 'other'
}
fuel_dict = {
    'gasoline': 'бензин',
    'diesel': 'дизель',
    'gas': 'газ',
    'hybrid-petrol': 'гибрид',
    'electric': 'электро',
    'hybrid-diesel': 'гибрид'
}
body_dict = {
    'внедорожник 5 дв.': 'suv',
    'седан': 'sedan',
    'лифтбек': 'liftback',
    'хэтчбек 5 дв.': 'hatchback',
    'универсал 5 дв.': 'universal',
    'универсал 3 дв.': 'universal',
    'минивэн': 'minivan',
    'купе': 'coupe',
    'компактвэн': 'minivan',
    'хэтчбек 3 дв.': 'hatchback',
    'пикап двойная кабина': 'pickup',
    'внедорожник 3 дв.': 'suv',
    'купе-хардтоп': 'coupe',
    'кабриолет': 'cabriolet',
    'фургон': 'van',
    'родстер': 'coupe',
    'микровэн': 'minivan',
    'седан-хардтоп': 'sedan',
    'пикап одинарная кабина': 'pickup',
    'пикап полуторная кабина': 'pickup',
    'внедорожник открытый': 'suv',
    'лимузин': 'limousine',
    'тарга': 'cabriolet',
    'седан 2 дв.': 'coupe',
    'фастбек': 'hatchback',
    'хэтчбек 4 дв.': 'hatchback',
    'фаэтон': 'cabriolet'
    
}
transmission_dict = {
    'автоматическая': 'automatic',
    'автомат': 'automatic',
    'механическая': 'mechanical',
    'механика': 'mechanical',
    'роботизированная': 'robot',
    'робот': 'robot',
    'вариатор': 'variator'
}
drivetrain_dict = {
    'полный': 'all',
    'передний': 'front',
    'задний': 'rear'
}

In [3]:
def clear_eng_power(hp):
    capacity = str(hp).split()[0]
    if capacity == 'undefined':
        return 2.0
    return float(capacity)

def eval_features(feature_string):
    feature_count = 0
    if len(feature_string) > 5:
        for i in eval(feature_string[2:-2]):
            feature_count += len(i['values'])
        return feature_count
    else:
        return 0

test['color'] = test['color'].apply(lambda x:color_dict[x])
#test['bodyType'] = test['bodyType'].apply(lambda x:body_dict[x])
test['vehicleTransmission'] = test['vehicleTransmission'].apply(lambda x:transmission_dict[x])
test['Привод'] = test['Привод'].apply(lambda x:drivetrain_dict[x])
test['productionDate'] = test['productionDate'].apply(lambda x:int(x))
test['mileage'] = test['mileage'].apply(lambda x:int(x))
test['enginePower'] = test['enginePower'].apply(clear_eng_power)
test['engineDisplacement'] = test['engineDisplacement'].apply(clear_eng_power)
test['features_count'] = (test['Комплектация'].apply(eval_features)//8).apply(lambda x: x if x<10 else 9)

test = test.rename(columns={'bodyType': 'body_type',
                            'brand': 'manufacturer_name',
                            'mileage': 'odometer_value',
                            'vehicleTransmission': 'transmission',
                            'productionDate': 'year_produced',
                            'fuelType': 'engine_fuel',
                            'engineDisplacement': 'engine_capacity',
                            'enginePower': 'engine_power',
                            'Привод': 'drivetrain'
})

test['year_produced'] = test['year_produced'] - 1938

In [4]:
autoru = autoru[autoru['manufacturer_name'].isin(test['manufacturer_name'])]

autoru['power_per_volume'] = autoru['engine_power']/autoru['engine_capacity']
test['power_per_volume'] = test['engine_power']/test['engine_capacity']

country_dict = {
'TOYOTA': 'JP',
'VOLKSWAGEN': 'DE',
'MERCEDES': 'DE',
'BMW': 'DE',
'NISSAN': 'JP',
'SKODA': 'CH',
'AUDI': 'DE',
'MITSUBISHI': 'JP',
'HONDA': 'JP',
'VOLVO': 'SW',
'LEXUS': 'JP',
'SUZUKI': 'JP',
'INFINITI': 'JP'
}

autoru['manufacturer_country'] = autoru['manufacturer_name'].replace(to_replace=country_dict)
test['manufacturer_country'] = test['manufacturer_name'].replace(to_replace=country_dict)

autoru['zero_odometer'] = autoru['odometer_value'].apply(lambda x: 1 if x==0 else 0)
test['zero_odometer'] = test['odometer_value'].apply(lambda x: 1 if x==0 else 0)

autoru = autoru[autoru['odometer_value']<autoru['odometer_value'].quantile(.97)]

autoru = autoru[autoru['engine_capacity'] < 7]

autoru = autoru[autoru['year_produced'] > 42]

In [5]:
from sklearn.model_selection import train_test_split
import numpy as np

features = ['body_type',#0
            'manufacturer_name',#1
            'transmission',#2
            'year_produced',#3
            'engine_fuel',#4
            'color',#5
            'drivetrain',#6
            'engine_capacity',#7
            'engine_power',#8
            'odometer_value',#9
            'power_per_volume',#10
            'manufacturer_country',#11
            'zero_odometer'#12
]

#cars = cars[features+['price']]
autoru = autoru[features+['price']]
#TRAIN = pd.concat([autoru, cars])

X = autoru[features]
y = autoru['price'].apply(np.log1p)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, shuffle=True, random_state=42)

In [ ]:
ITERATIONS = 5000
LR         = 0.2

from catboost import CatBoostRegressor
model = CatBoostRegressor(iterations = ITERATIONS,
                          learning_rate = LR,
                          #random_seed = 42,
                          #depth = 6,
                          #l2_leaf_reg = 1,
                          eval_metric = 'MAPE',
                          custom_metric = ['R2', 'MAE'],
                          early_stopping_rounds = 500
                         )

In [ ]:
model.fit(X_train, y_train,
            cat_features=[0,1,2,4,5,6,11],
            eval_set=(X_test, y_test),
            verbose_eval=500,
            use_best_model=True,
            plot=True
            )

In [ ]:
model.get_feature_importance(prettified=True)

In [ ]:
test_pred = test[features]
pred = model.predict(test_pred)

In [ ]:
sample_submission = pd.DataFrame()
sample_submission['price'] = pred
sample_submission['price'] = sample_submission['price'].apply(np.expm1)
sample_submission = sample_submission.reset_index().rename(columns={'index': 'id'})
sample_submission.to_csv(f'submission_v08.csv', index=False)
sample_submission.head()

In [ ]:
random_car = ['хетчбэк 3 дв.', 'HONDA', 'variator', 2012-1938, 'бензин', 'gray', 'all', 1.5, 88, 126089, 88/1.5, 'JP']
price = model.predict(random_car)
np.expm1(price)